In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import torch
from torch import nn

In [2]:
learning_rate = 0.01
batch_size = 1
epochs = 400

### read data

In [3]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [4]:
train.head()

,row_id,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
0,0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
1,1,0,0,0,520,2015,1,1,1,3,31,1,1,3,0
2,2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
3,3,0,1,1,572,2015,1,1,1,3,31,1,1,3,0
4,4,0,1,0,911,2015,1,1,1,3,31,1,1,3,0


### data preprocessing

In [5]:
train = train.set_index('row_id')
test = test.set_index('row_id')


In [6]:
train = train[train['num_sold']<500]
# train = train[train['num_sold']>50]
# train = train[['country', 'store', 'product', 'month', 'year', 'num_sold']]

In [7]:
train

,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
row_id,,,,,,,,,,,,,,
0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
5,0,1,2,283,2015,1,1,1,3,31,1,1,3,0
8,1,0,2,250,2015,1,1,1,3,31,1,1,3,0
11,1,1,2,395,2015,1,1,1,3,31,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26282,0,0,2,238,2018,4,12,31,0,31,365,1,0,0
26285,0,1,2,360,2018,4,12,31,0,31,365,1,0,0
26288,1,0,2,351,2018,4,12,31,0,31,365,1,0,0


In [8]:
x_data = train.drop('num_sold', axis=1)
# y_data = train.num_sold
# std = StandardScaler()
# x_data = std.fit_transform(x_data)
# test = std.transform(test)

test = torch.tensor(test.values)
y_data = train[['num_sold']]
x_data = torch.tensor(x_data.values)
y_data = torch.tensor(y_data.values)

In [9]:
x_data

tensor([[0, 0, 1,  ..., 1, 3, 0],
        [0, 0, 2,  ..., 1, 3, 0],
        [0, 1, 2,  ..., 1, 3, 0],
        ...,
        [1, 0, 2,  ..., 1, 0, 0],
        [2, 0, 2,  ..., 1, 0, 0],
        [2, 1, 2,  ..., 1, 0, 0]])

In [10]:
x_data = torch.tensor(x_data, dtype=torch.float32)
test = torch.tensor(test, dtype=torch.float32)

y_data = torch.tensor(y_data, dtype=torch.float32)

C:\Users\willi\AppData\Local\Temp\ipykernel_768\1023721312.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_data = torch.tensor(x_data, dtype=torch.float32)
C:\Users\willi\AppData\Local\Temp\ipykernel_768\1023721312.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test = torch.tensor(test, dtype=torch.float32)
C:\Users\willi\AppData\Local\Temp\ipykernel_768\1023721312.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_data = torch.tensor(y_data, dtype=torch.float32)


In [11]:
print(x_data[:10])

print(x_data.shape)

tensor([[0.0000e+00, 0.0000e+00, 1.0000e+00, 2.0150e+03, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 3.0000e+00, 3.1000e+01, 1.0000e+00, 1.0000e+00, 3.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.0000e+00, 2.0150e+03, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 3.0000e+00, 3.1000e+01, 1.0000e+00, 1.0000e+00, 3.0000e+00,
         0.0000e+00],
        [0.0000e+00, 1.0000e+00, 2.0000e+00, 2.0150e+03, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 3.0000e+00, 3.1000e+01, 1.0000e+00, 1.0000e+00, 3.0000e+00,
         0.0000e+00],
        [1.0000e+00, 0.0000e+00, 2.0000e+00, 2.0150e+03, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 3.0000e+00, 3.1000e+01, 1.0000e+00, 1.0000e+00, 3.0000e+00,
         0.0000e+00],
        [1.0000e+00, 1.0000e+00, 2.0000e+00, 2.0150e+03, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 3.0000e+00, 3.1000e+01, 1.0000e+00, 1.0000e+00, 3.0000e+00,
         0.0000e+00],
        [2.0000e+00, 0.0000e+00, 1.0000e+00, 2.0150e+03, 1.0000e+00, 1.0000e+00,

### model train

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

In [13]:
# np.unique(y_train.detach().numpy())

In [14]:
from torch.utils.data import TensorDataset

In [15]:
from torch.utils.data import DataLoader

In [16]:
dataset_train = TensorDataset(x_train, y_train)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
loader_test = DataLoader(dataset_test, batch_size=256, shuffle=True)


In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [18]:
import torch.nn.functional as F
n_feature = 13
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(n_feature, n_feature*2)
        self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        self.bn1 = nn.BatchNorm1d(n_feature*4)
        self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        self.fc4 = nn.Linear(n_feature*8, n_feature*16)
        # self.bn2 = nn.BatchNorm1d(n_feature*16)
        # self.fc5 = nn.Linear(n_feature*16, n_feature*32)
        # self.fc6 = nn.Linear(n_feature*32, n_feature*64)
        # self.fc7 = nn.Linear(n_feature*64, n_feature*128)
        # self.bn3 = nn.BatchNorm1d(n_feature*128)
        # self.fc8 = nn.Linear(n_feature*128, n_feature*64)
        # self.fc9 = nn.Linear(n_feature*64, n_feature*32)
        # self.fc10 = nn.Linear(n_feature*32, n_feature*16)
        # self.bn4 = nn.BatchNorm1d(n_feature*16)
        # self.fc11 = nn.Linear(n_feature*16, n_feature*8)
        # self.fc12 = nn.Linear(n_feature*8, n_feature*4)
        # self.bn5 = nn.BatchNorm1d(n_feature*4)
        # self.fc13 = nn.Linear(n_feature*4, n_feature*2)
        # self.fc14 = nn.Linear(n_feature*2, n_feature)
        self.fc15 = nn.Linear(n_feature*16, 1)

        # self.fc1 = nn.Linear(n_feature, n_feature*2)
        # self.bn1 = nn.BatchNorm1d(n_feature*2)
        # self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        # self.bn2 = nn.BatchNorm1d(n_feature*4)
        # self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        # self.bn3 = nn.BatchNorm1d(n_feature*8)
        # self.fc4 = nn.Linear(n_feature*8, n_feature*4)
        # self.fc5 = nn.Linear(n_feature*4, n_feature*2)
        # self.bn5 = nn.BatchNorm1d(n_feature*2)
        # self.fc6 = nn.Linear(n_feature*2, n_feature)
        # self.fc7 = nn.Linear(n_feature,  1)


    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(self.fc2(x))
        x = self.bn1(x)

        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        # x = self.bn2(x)
        # x = F.relu(self.fc5(x)) 
        # x = F.relu(self.fc6(x) )
        # x = F.relu(self.fc7(x) )
        # x = self.bn3(x) 
        # x = F.relu(self.fc8(x) )
        # x = F.relu(self.fc9(x) )
        # x = F.relu(self.fc10(x) )
        # x = self.bn4(x)
        # x = F.relu(self.fc11(x) )
        # x = F.relu(self.fc12(x) )
        # x = self.bn5(x) 
        # x = F.relu(self.fc13(x) )
        # x = F.relu(self.fc14(x))
        x = self.fc15(x) 
        
        # x = self.fc1(x)
        # x = F.relu(self.bn1(x))
        # x = F.relu(self.fc2(x))
        # x= F.relu(self.bn2(x) )
        # x= F.relu(self.fc3(x) )
        # x= F.relu(self.bn3(x) )
        # x= F.relu(self.fc4(x) )
        # x = F.relu(self.fc5(x))
        # x = F.relu(self.bn5(x))
        # x = F.relu(self.fc6(x))
        # x = self.fc7(x)
        # x = self.fc8(x)


        
        return x

In [19]:
model = NeuralNetwork().to(device)
# model.apply(init_weights)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=13, out_features=26, bias=True)
  (fc2): Linear(in_features=26, out_features=52, bias=True)
  (bn1): BatchNorm1d(52, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=52, out_features=104, bias=True)
  (fc4): Linear(in_features=104, out_features=208, bias=True)
  (fc15): Linear(in_features=208, out_features=1, bias=True)
)


In [20]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        # scheduler.step()
        optimizer.step()

    loss = loss.item()
    loss = loss ** 0.5
    print(f"loss: {loss:>7f}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    test_loss = test_loss ** 0.5
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [21]:
from math import exp
import math

In [22]:
def cyclical_lr(stepsize, min_lr=3e-4, max_lr=3e-3):
    
    # Scaler: we can adapt this if we do not want the triangular CLR
    scaler = lambda x: 1.

    # Lambda function to calculate the LR
    lr_lambda = lambda it: min_lr + (max_lr - min_lr) * relative(it, stepsize)

    # Additional function to see where on the cycle we are
    def relative(it, stepsize):
        cycle = math.floor(1 + it / (2 * stepsize))
        x = abs(it / stepsize - 2 * cycle + 1)
        return max(0, (1 - x)) * scaler(cycle)

    return lr_lambda

In [23]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
step_size = 4*len(dataset_train)
clr = cyclical_lr(step_size)
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, [clr])
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader_train, model, loss_fn, optimizer)
    test_loop(loader_test, model, loss_fn)
    # optimizer.lr = exp_decay(t)
print("Done!")

Epoch 1
-------------------------------
loss: 15.774013
Test Error: 
 Avg loss: 15.300975 

Epoch 2
-------------------------------
loss: 10.206771
Test Error: 
 Avg loss: 10.256464 

Epoch 3
-------------------------------
loss: 9.912325
Test Error: 
 Avg loss: 10.298994 

Epoch 4
-------------------------------
loss: 9.106332
Test Error: 
 Avg loss: 10.592042 

Epoch 5
-------------------------------
loss: 9.633043
Test Error: 
 Avg loss: 9.761942 

Epoch 6
-------------------------------
loss: 10.721990
Test Error: 
 Avg loss: 9.682378 

Epoch 7
-------------------------------
loss: 10.020507
Test Error: 
 Avg loss: 9.914767 

Epoch 8
-------------------------------
loss: 9.445580
Test Error: 
 Avg loss: 10.257036 

Epoch 9
-------------------------------
loss: 10.986398
Test Error: 
 Avg loss: 9.892687 

Epoch 10
-------------------------------
loss: 9.109309
Test Error: 
 Avg loss: 9.748711 

Epoch 11
-------------------------------
loss: 8.571415
Test Error: 
 Avg loss: 9.919763 

In [24]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)


In [25]:
x_train = x_train.to(device)
x_test = x_test.to(device)

train_pred = model(x_train)
test_pred = model(x_test)

train_score = SMAPE(y_train.cpu().detach().numpy(), train_pred.cpu().detach().numpy())
test_score = SMAPE(y_test.cpu().detach().numpy(), test_pred.cpu().detach().numpy())

print('train_score ', train_score, 'test_score', test_score)

train_score  24.077213 test_score 25.317478


In [26]:
test = test.to(device)

In [27]:
test_pred = model(test)

In [28]:
train_pred.cpu()

tensor([[124.4429],
        [ 87.0698],
        [214.4630],
        ...,
        [148.6111],
        [ 90.1939],
        [109.9326]], grad_fn=<ToCopyBackward0>)

In [29]:
# unique, counts = np.unique(y_train.cpu().detach().numpy(), return_counts=True)
# unique_p, counts_p = np.unique(np.round(train_pred.cpu().detach().numpy()), return_counts=True)
# d_p = pd.DataFrame()
# d_p.index = unique_p
# d_p['count_pred'] = counts_p
# d = pd.DataFrame()
# d.index = unique
# d['count_true'] = counts
# d1 = d.join(d_p)
# d1 = d1.fillna(0)


In [33]:
unique, counts = np.unique(y_train.cpu().detach().numpy(), return_counts=True)
unique_p, counts_p = np.unique(np.round(train_pred.cpu().detach().numpy()), return_counts=True)
d_p = pd.DataFrame(index = unique_p,data = counts_p,columns=['count_pred'])
d = pd.DataFrame(index= unique,data=  counts,columns=['count_true'])
d1 = d.join(d_p)
d1 = d1.fillna(0)

In [34]:
d_p

,count_pred
80.0,7
81.0,11
82.0,25
83.0,36
84.0,46
...,...
414.0,1
416.0,1
417.0,1
420.0,1


In [35]:
import plotly.express as px

fig = px.line(d1, x= d.index, y = ['count_true', 'count_pred'])
fig.show()

In [32]:
submission = pd.read_csv('sample_submission.csv')
submission.num_sold = test_pred.cpu().detach().numpy().reshape(-1)
submission.to_csv('submission.csv', index=False)